In [1]:
!pip install sdv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts')
import my_utils

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import my_utils
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup
import numpy as np
import os
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
data_folder = "../data"
output_folder = "../output"


# Check if the data folder exists, if not, create it
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    print(f"Created folder: {data_folder}")
else:
    print(f"Folder already exists: {data_folder}")
# Check if the output folder exists, if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created folder: {output_folder}")
else:
    print(f"Folder already exists: {output_folder}")

Folder already exists: ../data
Folder already exists: ../output


R Code to get the First Data set

In [ ]:
data_for_cleaning_to_start = pd.read_csv('../data/college_drafted_right.csv')

## Data Scraping Code

## Getting College Stats

In [ ]:
college_data = my_utils.get_college_stats(data_for_cleaning_to_start)

## Getting Strength of Schedule by Season (Scraping)

Note, I used the following links to scrape the SOS for each season. Because the links were not always in HTML table form, I found it easy to copy and paste each table from the website into excel. I then saved each of this files as SOS_{year}.csv in my data folder.

1) https://warrennolan.com/baseball/2011/sos

2) https://warrennolan.com/baseball/2012/sos

3) https://warrennolan.com/baseball/2013/sos

4) https://warrennolan.com/baseball/2014/sos

5) https://warrennolan.com/baseball/2015/sos

6) https://warrennolan.com/baseball/2016/sos

7) https://warrennolan.com/baseball/2017/sos

8) https://warrennolan.com/baseball/2018/sos

9) https://warrennolan.com/baseball/2019/sos

In [ ]:
college_data = pd.read_csv('../output/college_batters_with_college_stats.csv')
college_data = college_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

In [ ]:
college_data_with_sos = my_utils.match_and_add_sos_info(college_data, sos_years=range(2011, 2020))
college_data_with_sos['SOS'] = college_data_with_sos['SOS'].fillna(0)
# Create the new 'Div1' column
college_data_with_sos['Div1'] = (college_data_with_sos['SOS'] > 0).astype(int)

In [ ]:
college_data_with_sos.to_csv('../output/college_data_with_sos.csv')

Note, this got information for all Division 1 Baseball Schools. Schools that do not have a SOS (which will be NAN) will be given an SOS of 0. I will also from this create a variable called Division 1 which will be 1 if the school has a SOS and 0 if not.

## Getting Position for Each Player (Scraping)

In [ ]:
college_data_with_sos= pd.read_csv('../output/college_data_with_sos.csv')
college_data_with_sos = college_data_with_sos.drop(columns=['Unnamed: 0'])

In [ ]:
college_data_with_sos_and_pos= my_utils.get_player_positions(college_data_with_sos)

In [ ]:
college_data_with_sos_and_pos.to_csv('../output/college_data_with_sos_and_pos.csv')

Replacing all NAN in SOS with a 0 and then make a new variable called Div1 which is 1 if SOS >0 and 0 if SOS ==0

## Phase 2: Row Selection, Feature Engineering, and Creating Balanced Dataset

START HERE!

In [4]:
college_data_with_sos_and_pos = pd.read_csv('../output/college_data_with_sos_and_pos.csv', )

In [5]:
train_college_data, test_college_data = train_test_split(college_data_with_sos_and_pos, test_size=0.2, random_state=42)

# Updated pipeline
phase_2_pipeline = Pipeline([
    ('phase2', my_utils.Phase2(
        balance_method='kmeans_sampling',
        response_column='make_mlb',
        binary=True,
        categorical_columns=['college', 'Div1', 'Position'],
        index_column='key_bbref_minors',
        drop_columns=['pro_played_first', 'col_played_last', 'birth_year', 'birth_month', 'birth_day', 'matched_college_name', 'Year'],
        k_means_to_consider=[5, 10, 20, 25, 50, 75, 100, 150, 200],
        columns_to_use_in_balancing=['college_WOBA', 'walk_to_strikeout', 'age', 'Div1_1', 'SOS'],
        columns_to_consider_for_dropping = {'Position': ['Relief', 'Starting', 'Pitcher']}, downsample = False,
        file_output = None
    ))

])

In [6]:
phase_2_train_before_balanced_train,dropped_indices= phase_2_pipeline.fit_transform(train_college_data)
phase_2_train_before_balanced_train.to_csv('../output/phase_2cleaned_but_not_balanced_training_data.csv')

Reindexing by key_bbref_minors
Dropping ['pro_played_first', 'col_played_last', 'birth_year', 'birth_month', 'birth_day', 'matched_college_name', 'Year'] as they are not useful for learning
Dropped 152 missing values
Dropped 15 rows from column Position based on values ['Relief', 'Starting', 'Pitcher']
Total dropped rows: 15 that had values in {'Position': ['Relief', 'Starting', 'Pitcher']}
Used One Hot Encoder with ['college', 'Div1', 'Position']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
phase_2_train_before_balanced_train['make_mlb'].value_counts()

0    3114
1     314
Name: make_mlb, dtype: int64

In [ ]:
balancing_pipeline = Pipeline([
    ('phase2', my_utils.Phase2(
        balance_method='kmeans_sampling',
        response_column='make_mlb',
        binary=True,
        random_state = 1,
        index_column='key_bbref_minors',
        k_means_to_consider=[5, 10, 20, 25, 50, 75, 100],
        columns_to_use_in_balancing=['college_WOBA', 'walk_to_strikeout', 'age', 'Div1_1', 'SOS']
        ,downsample = True,
        file_output = '../output/phase_2_training_data.csv',
        dict_output = '../output/phase_2_downsampling_dict.txt'
    ))

])


phase_2_train_balanced, balanced_dict = balancing_pipeline.fit_transform(phase_2_train_before_balanced_train)

Dropped 0 missing values
Total dropped rows: 0 that had values in {}
Doing kmeans clustering with 314 clusters
Doing kmeans clustering with 5 clusters
Doing kmeans clustering with 10 clusters
Doing kmeans clustering with 20 clusters
Doing kmeans clustering with 25 clusters
Doing kmeans clustering with 50 clusters
Doing kmeans clustering with 75 clusters
Doing kmeans clustering with 100 clusters
Using kmeans with 50 clusters (equal method) to downsample for balanced data as it has the best combination of high mean score and low std
Sending score_dict to ../output/phase_2_downsampling_dict.txt
Sending data to ../output/phase_2_training_data.csv


## Phase 3: Feature Selection

In [7]:
phase_2_train_balanced = pd.read_csv('../output/phase_2_training_data.csv', index_col = 0)

In [ ]:
phase_2_train_balanced.loc[phase_2_train_balanced['Div1_1'] == 1,]['make_mlb'].value_counts()

1    303
0    239
Name: make_mlb, dtype: int64

In [ ]:
phase_2_train_balanced.loc[phase_2_train_balanced['Div1_1'] == 0,]['make_mlb'].value_counts()

0    75
1    11
Name: make_mlb, dtype: int64

In [8]:
numeric_columns = ['college_WOBA', 'walk_to_strikeout', 'age', 'SB', 'SOS']
response_variable = 'make_mlb'
warnings.filterwarnings("ignore", message="Features .* are constant.", category=UserWarning)
k_best_options = [1, 3, 5, 6,7,8,9,10,15]
# Step 2: Create a pipeline
phase_3_pipeline = Pipeline([
    ('phase3', my_utils.Phase3(
    scale_normalize=True,
    k_to_consider=k_best_options,
    response_variable= response_variable,
    classification=True,
    regression=False,
    random_state = 42,
    numeric_columns = numeric_columns,
    pca_params= None,
    file_output = '../output/phase_3_training_data.csv',
    dict_output = '../output/phase_3_score_dict.txt')
)
])

In [9]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
(phase_3_training_data,
 columns_for_phase_4,
 phase_3_score_dict,
 phase_3_scaler,
 phase_3_normalizer) = phase_3_pipeline.fit_transform(phase_2_train_balanced)

Performing select 1 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 3 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 5 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 6 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 7 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 8 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 9 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 10 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 15 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 1 best with StandardScaler() scaler and None normalizer
Performing select 3 best with StandardScaler() scaler and None normalizer
Performing select 5 best with StandardScaler() scaler and None normalizer
Performing select 6 

In [ ]:
my_utils.boxplots_for_numerics_to_pdf(
    phase_2_train_balanced,
    columns=['college_WOBA', 'age', 'SOS', 'college_Arizona', 'college_Clemson',
             'college_Florida', 'college_Louisiana\xa0State', 'Div1_1'],
    pdf_filename='../output/boxplot_for_numeric_variables_chosen.pdf'
)

## Phase 4: Modeling

In [ ]:
phase_3_training_data = pd.read_csv('../output/phase_3_training_data.csv', index_col = 0)

In [ ]:
my_utils.create_boxplots_and_save_to_pdf(phase_2_train_balanced, phase_3_training_data, '../output/boxplots_for_scaled_and_unscaled.pdf')

Creating BoxPlot for college_WOBA
Creating BoxPlot for age
Creating BoxPlot for SOS
Sent to ../output/boxplots_for_scaled_and_unscaled.pdf


In [ ]:
param_grid_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

param_grid_logistic = {
    'penalty': ['l1'],
    'C': [0.01, 0.1, 1.0, 10.0],
    'solver': ['liblinear']  # specify 'liblinear' solver for 'l1' penalty
}
param_grid_svm = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf']
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance']
}

# Create and test each classifier
# Usage
# Usage
classifier_param_tuples = [
    (DecisionTreeClassifier(), param_grid_tree),
    (RandomForestClassifier(), param_grid_rf),
    (XGBClassifier(), param_grid_xgb),
    (LogisticRegression(), param_grid_logistic),
    (SVC(), param_grid_svm),
    (KNeighborsClassifier(), param_grid_knn)
]




phase_4_pipeline = Pipeline([
    ('phase4', my_utils.Phase4('make_mlb', classifier_param_tuples, dict_output = '../output/modeling_scores.txt', model_output_path = '../output/model.pkl'))
])

# Assuming phase_3_training_data is your dataset
best_classifier, best_score, performance_grid = phase_4_pipeline.fit_transform(phase_3_training_data)

{'mean_accuracy': 0.6895619047619046, 'std_accuracy': 0.027542514652418443, 'training_time': 3.310356855392456, 'best_params': {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 1}}
Mean score is 0.6895619047619046 and baseline is 0.7436571428571428
Confidence interval for difference between  mean score and baseline is [-0.08392500493026556, -0.024265471260210843]
DecisionTreeClassifier does not outperform baseline
{'mean_accuracy': 0.7437333333333334, 'std_accuracy': 0.038687847822811205, 'training_time': 165.29605865478516, 'best_params': {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2}}
Mean score is 0.7437333333333334 and baseline is 0.7436571428571428
Confidence interval for difference between  mean score and baseline is [-0.038093924306172415, 0.03824630525855354]
RandomForestClassifier does not outperform baseline
{'mean_accuracy': 0.7357714285714285, 'std_accuracy': 0.053583922258600765, 'training_time': 

## Phase 5: Evaluation

In [10]:
import pickle
pickle_file_path = '../output/model.pkl'

# Load the model from the file
with open(pickle_file_path, 'rb') as file:
    best_classifier = pickle.load(file)

In [11]:
intercept = best_classifier.intercept_[0]
coefficients = best_classifier.coef_[0]
features = phase_3_training_data.columns


print("Coefficients for each variable:")
for feature, coef in zip(features, coefficients):
    print(f"{feature}: {coef}")

Coefficients for each variable:
college_WOBA: 0.831556935280601
age: -0.8602733829777233
SOS: 1.1886583957249541
college_Arizona: 0.8337605152977898
college_Clemson: 3.68417472349427
college_Florida: 2.740913024070358
college_Louisiana State: 4.226696329955828
Div1_1: -6.580589662967994


In [12]:
(phase_2_test_data, test_phase_2_dropped_indices) = phase_2_pipeline.transform(test_college_data)


Reindexing by key_bbref_minors
Dropping ['pro_played_first', 'col_played_last', 'birth_year', 'birth_month', 'birth_day', 'matched_college_name', 'Year'] as they are not useful for learning
Dropped 32 missing values
Dropped 6 rows from column Position based on values ['Relief', 'Starting', 'Pitcher']
Total dropped rows: 6 that had values in {'Position': ['Relief', 'Starting', 'Pitcher']}
Used One Hot Encoder with ['college', 'Div1', 'Position']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
numeric_columns = ['college_WOBA', 'walk_to_strikeout', 'age', 'SB', 'SOS']
# Preparation of test data for phase 3
phase_3_test_data = my_utils.prepare_test_data_for_phase_4(
    data=phase_2_test_data,
    response_var='make_mlb',
    numeric_columns=numeric_columns,
    scaler=phase_3_scaler,
    normalizer=phase_3_normalizer,
    pca_model=None,
    selected_features=columns_for_phase_4,
    file_output = '../output/test_data_for_modeling.csv')

Sending data to ../output/test_data_for_modeling.csv


In [ ]:
original_data_path = '../output/college_data_with_sos_and_pos.csv'
final_prediction_test_set = my_utils.predict_with_dropped_indices(phase_3_test_data, model = best_classifier,
                                      original_data_path = original_data_path,
                                      dropped_indices = test_phase_2_dropped_indices,
                                    file_output = '../output/predictions_on_test_set.csv')

Sending data to ../output/predictions_on_test_set.csv


## Phase 5: Determining Best Threshold

In [ ]:
thresholds = np.arange(0.1, 1, 0.01)
best_threshold, best_gmean, best_sensitivity, best_specificity = my_utils.find_best_threshold(final_prediction_test_set, 'make_mlb', 'predicted_probability', thresholds)


Best Threshold: 0.4999999999999998
Geometric Mean of Sensitivity and Specificity: 0.81
Sensitivity (True Positive Rate): 0.84 - This model correctly predicts 83.51% of those who made it to the MLB.
Specificity (True Negative Rate): 0.79 - This model correctly predicts 78.84% of those who did not make it to the MLB.


## Phase 5: Ablation Test

In [ ]:
phase_3_training_data = pd.read_csv('../output/phase_3_training_data.csv', index_col = 0)

In [ ]:
performance_without_dif_variables =  my_utils.perform_ablation_test_classification(
    training_data_for_model = phase_3_training_data,
    test_data_for_model = phase_3_test_data,
    predictor = best_classifier,
    response_var = 'make_mlb',
    original_data_path = original_data_path,
    original_geo_mean = best_gmean,
    dropped_indices = test_phase_2_dropped_indices,
    col_to_consider = columns_for_phase_4,
    file_output = '../output/abalation_test.csv',
    classification = True,
    regression = False
)

Dropping college_WOBA from both training and test set
Fitting LogisticRegression(C=10.0, penalty='l1', solver='liblinear') to train data
Best Threshold: 0.4999999999999998
Geometric Mean of Sensitivity and Specificity: 0.79
Sensitivity (True Positive Rate): 0.82 - This model correctly predicts 82.47% of those who made it to the MLB.
Specificity (True Negative Rate): 0.75 - This model correctly predicts 74.92% of those who did not make it to the MLB.
Dropping age from both training and test set
Fitting LogisticRegression(C=10.0, penalty='l1', random_state=42, solver='liblinear') to train data


/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({
/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


Best Threshold: 0.47999999999999976
Geometric Mean of Sensitivity and Specificity: 0.75
Sensitivity (True Positive Rate): 0.79 - This model correctly predicts 79.38% of those who made it to the MLB.
Specificity (True Negative Rate): 0.72 - This model correctly predicts 71.52% of those who did not make it to the MLB.
Dropping SOS from both training and test set
Fitting LogisticRegression(C=10.0, penalty='l1', random_state=42, solver='liblinear') to train data
Best Threshold: 0.47999999999999976
Geometric Mean of Sensitivity and Specificity: 0.80
Sensitivity (True Positive Rate): 0.86 - This model correctly predicts 85.57% of those who made it to the MLB.
Specificity (True Negative Rate): 0.76 - This model correctly predicts 75.54% of those who did not make it to the MLB.
Dropping college_Arizona from both training and test set
Fitting LogisticRegression(C=10.0, penalty='l1', random_state=42, solver='liblinear') to train data


/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({
/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


Best Threshold: 0.4999999999999998
Geometric Mean of Sensitivity and Specificity: 0.80
Sensitivity (True Positive Rate): 0.82 - This model correctly predicts 82.47% of those who made it to the MLB.
Specificity (True Negative Rate): 0.79 - This model correctly predicts 78.53% of those who did not make it to the MLB.
Dropping college_Clemson from both training and test set
Fitting LogisticRegression(C=10.0, penalty='l1', random_state=42, solver='liblinear') to train data
Best Threshold: 0.4999999999999998
Geometric Mean of Sensitivity and Specificity: 0.81
Sensitivity (True Positive Rate): 0.84 - This model correctly predicts 83.51% of those who made it to the MLB.
Specificity (True Negative Rate): 0.79 - This model correctly predicts 78.53% of those who did not make it to the MLB.
Dropping college_Florida from both training and test set
Fitting LogisticRegression(C=10.0, penalty='l1', random_state=42, solver='liblinear') to train data


/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({
/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


Best Threshold: 0.4999999999999998
Geometric Mean of Sensitivity and Specificity: 0.81
Sensitivity (True Positive Rate): 0.84 - This model correctly predicts 83.51% of those who made it to the MLB.
Specificity (True Negative Rate): 0.79 - This model correctly predicts 78.64% of those who did not make it to the MLB.
Dropping college_Louisiana State from both training and test set
Fitting LogisticRegression(C=10.0, penalty='l1', random_state=42, solver='liblinear') to train data
Best Threshold: 0.5099999999999998
Geometric Mean of Sensitivity and Specificity: 0.80
Sensitivity (True Positive Rate): 0.81 - This model correctly predicts 81.44% of those who made it to the MLB.
Specificity (True Negative Rate): 0.79 - This model correctly predicts 79.05% of those who did not make it to the MLB.
Dropping Div1_1 from both training and test set
Fitting LogisticRegression(C=10.0, penalty='l1', random_state=42, solver='liblinear') to train data
Best Threshold: 0.4999999999999998
Geometric Mean of 

/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({
/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


## Refitting Model Based On Ablation Test and Getting Information on Misclassified Test Points

In [14]:
import pickle
pickle_file_path = '../output/model.pkl'

# Load the model from the file
with open(pickle_file_path, 'rb') as file:
    best_classifier = pickle.load(file)
(phase_2_test_data, test_phase_2_dropped_indices) = phase_2_pipeline.transform(test_college_data)


Reindexing by key_bbref_minors
Dropping ['pro_played_first', 'col_played_last', 'birth_year', 'birth_month', 'birth_day', 'matched_college_name', 'Year'] as they are not useful for learning
Dropped 32 missing values
Dropped 6 rows from column Position based on values ['Relief', 'Starting', 'Pitcher']
Total dropped rows: 6 that had values in {'Position': ['Relief', 'Starting', 'Pitcher']}
Used One Hot Encoder with ['college', 'Div1', 'Position']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
phase_3_training_data = pd.read_csv('../output/phase_3_training_data.csv', index_col = 0)
thresholds = np.arange(0.1, 1, 0.01)
original_data_path = '../output/college_data_with_sos_and_pos.csv'
new_misclassified_df, final_prediction_test_set, best_threshold, best_gmean, best_sensitivity, best_specificity, new_model  = my_utils.refit_model_and_get_new_misclassified(
    predictor = best_classifier,
    columns_to_drop = ['Div1_1'],
    response_col = 'make_mlb',
    prob_col = 'predicted_probability',
    pdf_output = '../output/misclassified_with_new_model.pdf',
    phase_2_test_data = phase_2_test_data,
    dropped_indices = test_phase_2_dropped_indices,
    phase_3_training_data = phase_3_training_data,
    thresholds = thresholds,
    phase_3_test_data = phase_3_test_data,
    file_output = '../output/new_predictions_df.csv',
    original_data_path = original_data_path,
    key = 'key_bbref_minors'

)

Fitting LogisticRegression(C=10.0, penalty='l1', solver='liblinear') to train data without ['Div1_1']
Sending data to ../output/new_predictions_df.csv
Best Threshold: 0.4999999999999998
Geometric Mean of Sensitivity and Specificity: 0.82
Sensitivity (True Positive Rate): 0.86 - This model correctly predicts 85.57% of those who made it to the MLB.
Specificity (True Negative Rate): 0.78 - This model correctly predicts 78.22% of those who did not make it to the MLB.
Returning new misclassifed df, new predicted set, new   best_threshold, best_gmean, best_sensitivity, best_specificity, new predictor


In [17]:
new_model.coef_

array([[ 0.8261162 , -0.89099156,  0.37110187,  1.090151  ,  4.10949926,
         3.49229887,  4.14530937]])

## Using GAN To Create Balanced DataSet

## IGNORE THIS CODE. NO RANDOM STATE FOR SDV LOAD THE DATA SET IN NEXT CHUNK

In [ ]:
input_path = '../output/phase_2cleaned_but_not_balanced_training_data.csv'
response_var = 'make_mlb'
output_path = '../output/balanced_synthetic_data.csv'
model_output_path = '../output/pickle_file_synthetic_creation.pkl'

balanced_data_with_gan_upsampling, synthasizer = my_utils.balance_and_generate_synthetic_data(
    input_path=input_path,
    response_var=response_var,
    output_path=output_path,
    per= 15,
    model_output_path=model_output_path,
    drop_columns_that_begin_with='college',
    col_to_keep = 'college_WOBA'
)

Dropped some columns for computational efficiency. Now left with 16 columns.
We have 314 in the minority class and 3114 in the majority class
Trying 44800 synthetic samples to generate at least 2800 synthetic data points with the minority label.
Fitting synthesizer
Generating synthetic data
We have 7535 data points in the minority class
Random Sampling 2800 from our 7535 aso we have class balance
Assinging identifiers for synthetic data points


In [ ]:
balanced_data_with_gan_upsampling['make_mlb'].value_counts()

0    3114
1    3114
Name: make_mlb, dtype: int64

## Phase 3 (Synthetic Data):

START HERE FOR SYNTHETIC DATA. RANDOM STATE DOES NOT WORK FOR SOME OF THE GAN PACKAGES

In [ ]:
balanced_data_with_gan_upsampling = pd.read_csv('../output/balanced_synthetic_data.csv', index_col =0)

In [ ]:
balanced_data_with_gan_upsampling['make_mlb'].value_counts()

0    3114
1    3114
Name: make_mlb, dtype: int64

In [ ]:
numeric_columns = ['college_WOBA', 'walk_to_strikeout', 'age', 'SB', 'SOS']
response_variable = 'make_mlb'
warnings.filterwarnings("ignore", message="Features .* are constant.", category=UserWarning)
k_best_options = [1, 3, 5, 6,7,8,9,10,15]
# Step 2: Create a pipeline
phase_3_pipeline_synthetic = Pipeline([
    ('phase3', my_utils.Phase3(
    scale_normalize=True,
    k_to_consider=k_best_options,
    response_variable= response_variable,
    classification=True,
    regression=False,
    random_state = 2,
    numeric_columns = numeric_columns,
    pca_params= None,
    file_output = '../output/phase_3_training_data_synthetic.csv',
    dict_output = '../output/phase_3_score_dict_synthetic.txt')
)
])

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
(phase_3_training_data_synthetic,
 columns_for_phase_4_synthetic,
 phase_3_score_dict_synthetic,
 phase_3_scaler_synthetic,
 phase_3_normalizer_synthetic) = phase_3_pipeline_synthetic.fit_transform(balanced_data_with_gan_upsampling)

Performing select 1 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 3 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 5 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 6 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 7 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 8 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 9 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 10 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 15 best with StandardScaler() scaler and MinMaxScaler() normalizer
Performing select 1 best with StandardScaler() scaler and None normalizer
Performing select 3 best with StandardScaler() scaler and None normalizer
Performing select 5 best with StandardScaler() scaler and None normalizer
Performing select 6 

In [ ]:
columns_for_phase_4_synthetic

Index(['college_WOBA', 'walk_to_strikeout', 'age', 'SB', 'SOS',
       'Position_Rightfielder', 'Position_Second'],
      dtype='object')

In [ ]:
my_utils.plot_data_with_synthetic_to_pdf(balanced_data_with_gan_upsampling, phase_3_training_data_synthetic, 'make_mlb', '../output/synthetic_versus_regular_data_graphs.pdf')

## Phase 4 (Synthetic Data): Modeling

In [ ]:
phase_3_training_data_synthetic = pd.read_csv('../output/phase_3_training_data_synthetic.csv', index_col =0)

param_grid_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

param_grid_logistic = {
    'penalty': ['l1'],
    'C': [0.01, 0.1, 1.0, 10.0],
    'solver': ['liblinear']  # specify 'liblinear' solver for 'l1' penalty
}
param_grid_svm = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf']
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance']
}

# Create and test each classifier
# Usage
# Usage
classifier_param_tuples = [
    (DecisionTreeClassifier(), param_grid_tree),
    (RandomForestClassifier(), param_grid_rf),
    (XGBClassifier(), param_grid_xgb),
    (LogisticRegression(), param_grid_logistic),
    (SVC(), param_grid_svm),
    (KNeighborsClassifier(), param_grid_knn)
]




phase_4_pipeline_synthetic = Pipeline([
    ('phase4', my_utils.Phase4('make_mlb', classifier_param_tuples, dict_output = '../output/modeling_scores_synthetic.txt',
                               model_output_path = '../output/model_synthetic.pkle'))
])


best_classifier_synthetic, best_score_synthetic, performance_grid_synthetic = phase_4_pipeline_synthetic.fit_transform(phase_3_training_data_synthetic)

{'mean_accuracy': 0.7447041456355116, 'std_accuracy': 0.011239007551447121, 'training_time': 12.699143171310425, 'best_params': {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 4}}
Mean score is 0.7447041456355116 and baseline is 0.6661857703687947
Confidence interval for difference between  mean score and baseline is [0.06272343571222777, 0.09431331482120603]
DecisionTreeClassifier outperforms baseline
Best classifier DecisionTreeClassifier is fitted with best parameters.
{'mean_accuracy': 0.7703939352917286, 'std_accuracy': 0.00897218864818141, 'training_time': 499.5179970264435, 'best_params': {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 4}}
Mean score is 0.7703939352917286 and baseline is 0.6661857703687947
Confidence interval for difference between  mean score and baseline is [0.08956987809352655, 0.11884645175234133]
RandomForestClassifier outperforms baseline
Best classifier RandomForestClassifier is fit

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

{'mean_accuracy': 0.669718875502008, 'std_accuracy': 0.009726735512695343, 'training_time': 2.6716394424438477, 'best_params': {'penalty': 'l1', 'C': 10.0, 'solver': 'liblinear'}}
Mean score is 0.669718875502008 and baseline is 0.669718875502008
Confidence interval for difference between  mean score and baseline is [-0.01205738625991986, 0.01205738625991986]
LogisticRegression does not outperform baseline
{'mean_accuracy': 0.7442192526123756, 'std_accuracy': 0.011182918047972376, 'training_time': 63.606940031051636, 'best_params': {'C': 10.0, 'kernel': 'rbf'}}
Mean score is 0.7442192526123756 and baseline is 0.669718875502008
Confidence interval for difference between  mean score and baseline is [0.06150905000262525, 0.08749170421810998]
SVC outperforms baseline
{'mean_accuracy': 0.7180485666582864, 'std_accuracy': 0.010422355953093675, 'training_time': 2.201761245727539, 'best_params': {'n_neighbors': 9, 'weights': 'uniform'}}
Mean score is 0.7180485666582864 and baseline is 0.6697188

## Phase 5 (Synthetic): Evaluation

In [ ]:
import pickle
pickle_file_path = '../output/model_synthetic.pkle'

# Load the model from the file
with open(pickle_file_path, 'rb') as file:
    best_classifier_synthetic = pickle.load(file)


In [ ]:
(phase_2_test_data, test_phase_2_dropped_indices) = phase_2_pipeline.transform(test_college_data)
numeric_columns = ['college_WOBA', 'walk_to_strikeout', 'age', 'SB', 'SOS']
# Preparation of test data for phase 3
phase_3_test_data_synthetic = my_utils.prepare_test_data_for_phase_4(
    data=phase_2_test_data,
    response_var='make_mlb',
    numeric_columns=numeric_columns,
    scaler=phase_3_scaler_synthetic,
    normalizer=phase_3_normalizer_synthetic,
    pca_model=None,
    selected_features=columns_for_phase_4_synthetic,
    file_output = '../output/test_data_for_modeling_synthetic.csv')

Reindexing by key_bbref_minors
Dropping ['pro_played_first', 'col_played_last', 'birth_year', 'birth_month', 'birth_day', 'matched_college_name', 'Year'] as they are not useful for learning
Dropped 32 missing values
Dropped 6 rows from column Position based on values ['Relief', 'Starting', 'Pitcher']
Total dropped rows: 6 that had values in {'Position': ['Relief', 'Starting', 'Pitcher']}
Used One Hot Encoder with ['college', 'Div1', 'Position']
Sending data to ../output/test_data_for_modeling_synthetic.csv


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
original_data_path = '../output/college_data_with_sos_and_pos.csv'
final_prediction_test_set_synthetic = my_utils.predict_with_dropped_indices(phase_3_test_data_synthetic, model = best_classifier_synthetic,
                                      original_data_path = original_data_path,
                                      dropped_indices = test_phase_2_dropped_indices,
                                    file_output = '../output/predictions_on_test_set_synthetic.csv')

Sending data to ../output/predictions_on_test_set_synthetic.csv


## Phase 5 (Synthetic): Determining Best Threshold

In [ ]:
thresholds = np.arange(0.1, 1, 0.01)
best_threshold_synthetic, best_gmean_synthetic, best_sensitivity_synthetic, best_specificity_synthetic = my_utils.find_best_threshold(final_prediction_test_set_synthetic, 'make_mlb', 'predicted_probability', thresholds)


Best Threshold: 0.24999999999999992
Geometric Mean of Sensitivity and Specificity: 0.74
Sensitivity (True Positive Rate): 0.92 - This model correctly predicts 91.75% of those who made it to the MLB.
Specificity (True Negative Rate): 0.60 - This model correctly predicts 60.47% of those who did not make it to the MLB.


In [ ]:
phase_3_training_data_synthetic = pd.read_csv('../output/phase_3_training_data_synthetic.csv', index_col = 0)

## Phase 5 (Synthetic): Ablation Test

In [ ]:
final_prediction_test_set_synthetic = pd.read_csv('../output/predictions_on_test_set_synthetic.csv', index_col = 0)
phase_3_training_data_synthetic = pd.read_csv('../output/phase_3_training_data_synthetic.csv', index_col = 0)
performance_without_dif_variables_synthetic =  my_utils.perform_ablation_test_classification(
    training_data_for_model = phase_3_training_data_synthetic,
    test_data_for_model = phase_3_test_data_synthetic,
    predictor = best_classifier_synthetic,
    response_var = 'make_mlb',
    original_data_path = original_data_path,
    original_geo_mean = best_gmean_synthetic,
    dropped_indices = test_phase_2_dropped_indices,
    col_to_consider = phase_3_training_data_synthetic.columns,
    file_output = '../output/results_of_ablation_synthetic.csv',
    classification = True,
    regression = False
)

Dropping college_WOBA from both training and test set
Fitting XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...) to train data
Best Threshold: 0.3199999999999999
Geometric Mean of Sensitivity and Specificity: 0.76
Sensitivity (True Positive Rate): 0.86 - This model correctly predicts 85.57% of those

/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


Best Threshold: 0.34999999999999987
Geometric Mean of Sensitivity and Specificity: 0.73
Sensitivity (True Positive Rate): 0.77 - This model correctly predicts 77.32% of those who made it to the MLB.
Specificity (True Negative Rate): 0.69 - This model correctly predicts 69.35% of those who did not make it to the MLB.
Dropping age from both training and test set
Fitting XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
    

/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


Best Threshold: 0.2599999999999999
Geometric Mean of Sensitivity and Specificity: 0.70
Sensitivity (True Positive Rate): 0.80 - This model correctly predicts 80.41% of those who made it to the MLB.
Specificity (True Negative Rate): 0.61 - This model correctly predicts 61.09% of those who did not make it to the MLB.
Dropping SB from both training and test set
Fitting XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
      

...

/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


Best Threshold: 0.23999999999999994
Geometric Mean of Sensitivity and Specificity: 0.74
Sensitivity (True Positive Rate): 0.93 - This model correctly predicts 92.78% of those who made it to the MLB.
Specificity (True Negative Rate): 0.59 - This model correctly predicts 59.03% of those who did not make it to the MLB.
Dropping Position_Second from both training and test set
Fitting XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraint

/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


Best Threshold: 0.3199999999999999
Geometric Mean of Sensitivity and Specificity: 0.76
Sensitivity (True Positive Rate): 0.87 - This model correctly predicts 86.60% of those who made it to the MLB.
Specificity (True Negative Rate): 0.67 - This model correctly predicts 67.39% of those who did not make it to the MLB.
Dropping make_mlb from both training and test set


/content/drive/MyDrive/ColabNotebooks/Final_Project/Scripts/my_utils.py:1346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_without_variables = performance_without_variables.append({


In [ ]:
performance_without_dif_variables_synthetic

,Variable,Original Geo Mean,New Geomean,Geomean Change
6,Position_Second,0.745,0.764,-0.019
0,college_WOBA,0.745,0.758,-0.013
3,SB,0.745,0.749,-0.005
5,Position_Rightfielder,0.745,0.740,0.005
1,walk_to_strikeout,0.745,0.732,0.013
4,SOS,0.745,0.711,0.034
2,age,0.745,0.701,0.044
